In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

## Tuning the hyper-parameters of your estimator
- Train models with different combinations of hyper parameters
- Use cross validation to evaluate the performance of models
- Choose the best one


In [3]:
# loading training/testing data
all_data = pd.read_csv("preprocessed_data.csv")
X = all_data.drop(["Survived"], axis=1)
y = all_data["Survived"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
print(len(X_train))
print(len(X_test))

712
179


In [5]:
model = RandomForestClassifier(max_depth=100, n_estimators=10)
model.fit(X_train, y_train)
predict_y = model.predict(X_train)
print("training accuracy:", (predict_y == y_train).sum()/ len(predict_y))


predict_y = model.predict(X_test)
print("testing accuracy:", (predict_y == y_test).sum()/ len(predict_y))

training accuracy: 0.976123595505618
testing accuracy: 0.7486033519553073


In [6]:
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.77653631, 0.78089888, 0.84831461, 0.76404494, 0.8258427 ])

You can see more classification metics here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [7]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 100,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [5,10,30,50,100], 'max_depth': [3,5,10,20]}]


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                   scoring='accuracy')
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_ )
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])
print(clf.cv_results_["params"])

{'max_depth': 10, 'n_estimators': 50}
0.8315571752191471
[0.81189796 0.80626416 0.82033882 0.81327686 0.82311632 0.81743327
 0.81613316 0.82031912 0.83013888 0.82871073 0.81460652 0.80904166
 0.82590367 0.83155718 0.82733182 0.80762336 0.81748252 0.82172757
 0.82029942 0.81469516]
[0.03364017 0.0316839  0.03927219 0.0263242  0.03320011 0.01973331
 0.03817921 0.03284027 0.03197497 0.03250415 0.01858558 0.01953519
 0.02564615 0.03802067 0.02702836 0.02069434 0.03069465 0.03838669
 0.03998483 0.02940191]
[{'max_depth': 3, 'n_estimators': 5}, {'max_depth': 3, 'n_estimators': 10}, {'max_depth': 3, 'n_estimators': 30}, {'max_depth': 3, 'n_estimators': 50}, {'max_depth': 3, 'n_estimators': 100}, {'max_depth': 5, 'n_estimators': 5}, {'max_depth': 5, 'n_estimators': 10}, {'max_depth': 5, 'n_estimators': 30}, {'max_depth': 5, 'n_estimators': 50}, {'max_depth': 5, 'n_estimators': 100}, {'max_depth': 10, 'n_estimators': 5}, {'max_depth': 10, 'n_estimators': 10}, {'max_depth': 10, 'n_estimators': 3

In [9]:
model = RandomForestClassifier(max_depth=5, n_estimators=30)
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.82681564, 0.81460674, 0.81460674, 0.79213483, 0.83707865])

In [10]:
print("Detailed classification report:") 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Detailed classification report:
              precision    recall  f1-score   support

           0       0.76      0.91      0.83       106
           1       0.81      0.59      0.68        73

    accuracy                           0.78       179
   macro avg       0.79      0.75      0.76       179
weighted avg       0.78      0.78      0.77       179


